In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

In [2]:
import nltk
from nltk import ngrams
from nltk.tokenize import word_tokenize
from collections import Counter
import time

In [3]:
data_comments_original =pd.read_csv('/content/drive/MyDrive/LLM_Project_DrFarhanAli_Tawseeq/Dataset/sgexams_comments_Oct2023.csv')
data_posts_original  = pd.read_csv('/content/drive/MyDrive/LLM_Project_DrFarhanAli_Tawseeq/Dataset/sgexams_posts_Oct2023.csv')

In [4]:
print(data_comments_original.columns)

Index(['Unnamed: 0', 'id', 'parent_id', 'body', 'link_id', 'created_utc',
       'author', 'score', 'is_op'],
      dtype='object')


In [5]:
selected_columns_data_posts_original = ['title', 'body']
reddit_posts = data_posts_original[selected_columns_data_posts_original]
reddit_comments = data_comments_original['body']

In [6]:
reddit_posts

,title,body
0,[A Levels] JC is starting soon for many of you...,"Hi, I understand many of you are feeling nervo..."
1,[o level] did anyone get a for art :/,so i got a b3 for art. :/ in the past few exam...
2,[Poly] Singapore Polytechnic DTVM EAE,Currently a 4E student taking O's this year. B...
3,[Poly] Ngee Ann Dragonboat,Hi just want to ask about your experiences as ...
4,[Uni] NUS Hall specifically kent ridge hall,ive been contemplating if i should stay in hal...
...,...,...
38915,[JC]MI VS YIJC,I kinda messed up my o levels and got 18 point...
38916,most difficult to understand emath topic? and ...,"hey guys, teacher here :) I just wanted to ask..."
38917,phys/chem science practical theory,"Hi guys, O level prelim practical is next week..."
38918,is pfp in my reach?,"I'll be honest, i didnt try at all for my prel..."


In [7]:
reddit_comments

0         hi! how many sci vs art stream classes r there...
1                     roughly whats the schedule on friday?
2                                                 [deleted]
3         Howâs the environment in RJC and the student...
4                               what do u think cop will be
                                ...                        
666973    Ohh I also have a friend with no o lvl phys gg...
666974                                 what is your rp/gpa?
666975    Yea like it depends some engine harder but the...
666976                                             78.75rp!
666977                                             Ohh icic
Name: body, Length: 666978, dtype: object

In [8]:
reddit_data = pd.concat([reddit_posts, reddit_comments], axis = 1)
reddit_data
#I dont kno why they arent concatenated into a single column

,title,body,body
0,[A Levels] JC is starting soon for many of you...,"Hi, I understand many of you are feeling nervo...",hi! how many sci vs art stream classes r there...
1,[o level] did anyone get a for art :/,so i got a b3 for art. :/ in the past few exam...,roughly whats the schedule on friday?
2,[Poly] Singapore Polytechnic DTVM EAE,Currently a 4E student taking O's this year. B...,[deleted]
3,[Poly] Ngee Ann Dragonboat,Hi just want to ask about your experiences as ...,Howâs the environment in RJC and the student...
4,[Uni] NUS Hall specifically kent ridge hall,ive been contemplating if i should stay in hal...,what do u think cop will be
...,...,...,...
666973,NaN,NaN,Ohh I also have a friend with no o lvl phys gg...
666974,NaN,NaN,what is your rp/gpa?
666975,NaN,NaN,Yea like it depends some engine harder but the...
666976,NaN,NaN,78.75rp!


In [9]:
n_order_unigrams = 1
n_order_bigrams = 2
n_order_trigrams = 3

In [10]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [13]:
#reddit_data = reddit_data.replace(np.nan, '', regex=True)
reddit_comments = ["" if pd.isna(comment) else comment for comment in reddit_comments]
tokenized_data = [word_tokenize(text.lower()) for text in reddit_comments]
bigram_data = [ngrams(tokens, n_order_bigrams) for tokens in tokenized_data]
trigram_data = [ngrams(tokens, n_order_trigrams) for tokens in tokenized_data]
unigram_data = [ngrams(tokens, n_order_unigrams) for tokens in tokenized_data]

In [14]:
flat_bigrams = [ngram for ngrams_list in bigram_data for ngram in ngrams_list]
flat_trigrams = [ngram for ngrams_list in trigram_data for ngram in ngrams_list]
flat_unigrams = [ngram for ngrams_list in unigram_data for ngram in ngrams_list]

In [ ]:
flat_bigrams

In [16]:
bigram_freq = nltk.FreqDist(flat_bigrams)
trigram_freq = nltk.FreqDist(flat_trigrams)

In [17]:
bigram_counts = len(flat_bigrams)
print(f"Number of {n_order_bigrams}-grams:")
print(f"Total number of unique {n_order_bigrams}-grams: {bigram_counts}")

trigram_counts = len(flat_trigrams)
print(f"Number of {n_order_trigrams}-grams:")
print(f"Total number of unique {n_order_trigrams}-grams: {trigram_counts}")

Number of 2-grams:
Total number of unique 2-grams: 26339273
Number of 3-grams:
Total number of unique 3-grams: 25691724


In [20]:
#print(f"{n_order_bigrams}-grams count:")
#for ngram, count in bigram_freq.items():
#    print(f"{ngram}: {count}")

In [21]:
#print(f"{n_order_trigrams}-grams count:")
#for ngram, count in trigram_freq.items():
#    print(f"{ngram}: {count}")

In [22]:
import sys
size_of_array = sys.getsizeof(flat_bigrams)
print(size_of_array)
size_of_array = sys.getsizeof(flat_trigrams)
print(size_of_array)

228599448
228599448


In [23]:
flat_bigrams[122]

('use', 'for')

In [25]:
#Here I am testing for a particular 1-gram but am not getting match
#target_2gram = ('will', 'do')
target_1gram = ('nus',)
start_time = time.time()
#dataset = reddit_comments
#target_bigram_count = sum(1 for tokens in flat_bigrams for ngram in ngrams(tokens, 2) if ngram == target_2gram)
count = 0
for i in range(len(flat_unigrams)):
  if flat_unigrams[i] == target_1gram:
    #print(i)
    count += 1

end_time = time.time()
print(f"Occurrences of the target 1-gram '{' '.join(target_1gram)}': {count}")
print(f"Time taken: {end_time - start_time} seconds")

Occurrences of the target 1-gram 'nus': 22975
Time taken: 2.368572950363159 seconds


In [26]:
#Here I am testing for a particular 2-gram
target_2gram = ('great','but')
#target_1gram = ('nus',)
start_time = time.time()
#dataset = reddit_comments
#target_bigram_count = sum(1 for tokens in flat_bigrams for ngram in ngrams(tokens, 2) if ngram == target_2gram)
count = 0
for i in range(len(flat_bigrams)):
  if flat_bigrams[i] == target_2gram:
    #print(i)
    count += 1

end_time = time.time()
print(f"Occurrences of the target 2-gram '{' '.join(target_2gram)}': {count}")
print(f"Time taken: {end_time - start_time} seconds")

Occurrences of the target 2-gram 'great but': 112
Time taken: 2.317117929458618 seconds
